Simulation for branching process with random mutation. Each time the pathogen transmits to a new person, it mutates. In this version, only beta mutates while gamma remains constant. 

We seed the population with an initial case (presumably spilled over from a reservoir). We branch until either the infection goes extinct, or it surpasses a threshold. In the latter case, we call it an outbreak. 

Notes: 
    1. Not sure how to compute interevent time correctly, but it doesn't matter in this version. 

Next steps: 
    1. Create graphs/visualizations (may need to figure out interevent time for this).
    2. Allow gamma to mutate.
    3. Include recurrent reservoir spillover. 
    4. Include backwards transmission.

In [72]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sympy as sm

In [73]:
#parameters
gamma = 0.1 #recovery rate - constant for everyone right now

#initial case(s)
infecteds = [[0.09, 0]] #1st elt is beta, 2nd elt is time infected

In [85]:
#returns 0 if extinction, 1 if outbreak
def sim(infecteds, gamma):
    
    max_infected = 100 #call it an outbreak if this many people are infected
    N_infected = len(infecteds) #number of infected people

    while True: 
        
        #print("\n", "infecteds now = ", infecteds)
        
        recovereds = [] #keep track of indices of recovered people
        
        for i, x in enumerate(infecteds): #i is index, x is value
            
            beta = x[0]
            t = x[1]

            #print("\n", "branching person", i)
            
            while True:
                
                dt = np.random.exponential(scale=1/(beta + gamma)) #interevent time (??)
                new_t = t + dt #time of new event
                
                #pick which new event, transmission or recovery, happens            
                ev = np.random.rand()
                prob_trans = beta / (beta+gamma) #probability of transmission instead of recovery

                if (ev < prob_trans): #transmission
                    
                    #print("person", i, " is transmitting")
                    
                    N_infected += 1
                    
                    #pick beta for new case
                    mut = np.random.normal(loc=0.0, scale=0.01, size=None) #mutation is a perturbation drawn from normal distribution
                    new_beta = max(0, beta + mut) #don't allow negative beta
                    
                    infecteds = np.append(infecteds, [[new_beta, new_t]], axis=0)
                    #print("new infection: ", [new_beta, new_t])

                else: #recovery
                    
                    #print("person", i, " recovers") 
                    
                    N_infected -= 1
                    
                    recovereds.append(i) #mark that they recovered
                    break #skip to next person

        #after each time we complete a round of branching everybody:
        #1. update the list of infecteds by deleting recovered people
        #2. and check if we have extinction/outbreak
        
        infecteds = np.delete(infecteds, recovereds, axis=0)
        
        if N_infected == 0:
            #print("\n", "extinction")
            return 0
        elif N_infected >= max_infected:
            #print("\n", "outbreak!")
            return 1

In [98]:
N_sims = 1000

N_outbreaks = 0
for i in range(N_sims):
    if sim(infecteds, gamma) == 1: N_outbreaks += 1
    
print("percentage of outbreaks = ", N_outbreaks/N_sims)

percentage of outbreaks =  0.046
